In [0]:
history()

In [1]:
int x=1;

1

In [1]:
histories()

In [1]:
HistoriesClient h = histories()

startup failed:
Script5.groovy: 1: unable to resolve class HistoriesClient 
 @ line 1, column 17.
   HistoriesClient h = histories()
                   ^

1 error


In [2]:
println "Hello World"

Cell returned null.

In [1]:
testString("antidote")

ANTIDOTE